<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Microsoft AutoGen - Kmeleon Demo

Microsoft AutoGen is a framework that enables the development of applications using large language models (LLMs) such as GPT-4. LLMs are powerful AI systems that can generate natural language texts for various tasks, such as writing, summarizing, translating, and answering questions. However, LLMs also have some limitations, such as being prone to errors, biases, and safety issues. Therefore, using LLMs effectively requires a lot of effort and expertise to design, implement, and optimize a workflow that can leverage their full potential.

AutoGen simplifies this process by allowing you to create customizable and conversable agents that can use LLMs, tools, and human inputs to solve tasks. These agents can communicate with each other through automated chat to coordinate their actions and exchange information. For example, you can create an agent that can write code based on natural language instructions, another agent that can check the code for errors and safety, and another agent that can execute the code and provide feedback. By automating chat among multiple capable agents, tasks can be performed autonomously or with human feedback, including tasks that require using tools via code.

AutoGen also provides a collection of working systems with different complexities that span a wide range of applications from various domains. These systems demonstrate how AutoGen can easily support diverse conversation patterns and workflows. Moreover, AutoGen provides an enhanced inference API that allows easy performance tuning, utilities like caching and error handling, and advanced usage patterns such as context programming and multi-config inference.

AutoGen is powered by collaborative research studies from Microsoft, Penn State University, and the University of Washington. You can learn more about AutoGen from the following sources:

Microsoft Research Blog: A blog post that introduces AutoGen and its features.

GitHub Repository: A repository that contains the source code, documentation, and examples of AutoGen, [here](https://microsoft.github.io/autogen).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
%pip install --quiet pyautogen~=0.1.0

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The json looks like the following:
```json
[
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key here>"
    },
    {
        "model": "gpt-4",
        "api_key": "<your Azure OpenAI API key here>",
        "api_base": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gpt-4-32k",
        "api_key": "<your Azure OpenAI API key here>",
        "api_base": "<your Azure OpenAI API base here>",
        "api_type": "azure",
        "api_version": "2023-06-01-preview"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.


In [ ]:
import autogen

llm_config={
    "request_timeout": 600,
    "seed": 42,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4"]},
    ),
    "temperature": 0,
}

## Example Task: Calls to APIs

We consider a scenario where one needs to interact with differents APIs

### Construct Agents

We create an assistant agent to solve tasks with coding and language skills. We create a user proxy agent to describe tasks and execute the code suggested by the assistant agent.

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    # the default system message of the AssistantAgent is overwritten here
    system_message="""
        You are a helpful AI assistant.
        You suggest coding and reasoning steps for another AI assistant to accomplish a task.
        For any action beyond writing code or reasoning, convert it to a step which can be implemented by writing code.
        For example, the action of browsing the web can be implemented by writing code which reads and prints the content of a web page.
        Finally, inspect the execution result.
        If the plan is not good, suggest a better plan.
        If the execution is wrong, analyze the error and suggest a fix.
        """,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
)

### Step-by-step Instructions

In [ ]:
task = """
You are going to follow the following sequence of tasks and do not forget the result of each of them if you need to use it in the next task:
1. Find the Bitcoin rate using this API: https://api.coindesk.com/v1/bpi/currentprice.json and determines if the rate is greater than 35000 or not.
2. According the above results if the rate of the Bitcoin is greater than 35000 then find a joke and tell me it in this API: https://official-joke-api.appspot.com/random_joke else find and choose a random activities from this API: https://www.boredapi.com/api/activity and tell me.
3. According to the above result oh the last question, find a phrase that is consistent with it and that makes sense.
"""

user_proxy.initiate_chat(assistant, message=task)

user_proxy (to assistant):


You are going to follow the following sequence of tasks and do not forget the result of each of them if you need to use it in the next task:
1. Find the Bitcoin rate using this API: https://api.coindesk.com/v1/bpi/currentprice.json and determines if the rate is greater than 35000 or not.
2. According the above results if the rate of the Bitcoin is greater than 35000 then find a joke and tell me it in this API: https://official-joke-api.appspot.com/random_joke else find and choose a random activities from this API: https://www.boredapi.com/api/activity and tell me.
3. According to the above result oh the last question, find a phrase that is consistent with it and that makes sense.


--------------------------------------------------------------------------------
assistant (to user_proxy):

Here are the steps to accomplish the tasks:

1. Write a function to fetch the Bitcoin rate from the API: https://api.coindesk.com/v1/bpi/currentprice.json. You can use the